In [43]:
#Courtesy: pFinTools.com

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select, WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import re, time

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

driver.implicitly_wait(1000)

driver.get("https://kuvera.in/portfolio")

driver.find_element(By.ID, "email").send_keys("user") #replace user with email id

driver.find_element(By.ID, "password").send_keys("pwd") #replace user with password

driver.find_element(By.ID, "login-btn").click()

driver.find_element(By.CSS_SELECTOR, ".b-portfolio-tab__tab-content").click()

driver.find_element(By.CSS_SELECTOR, ".b-portfolio-tab__value--selected").click()

driver.find_element(By.CSS_SELECTOR, ".default-message:nth-child(3)").click()

driver.find_element(By.CSS_SELECTOR, ".b-fund-sort-wrapper__bottom-sheet-sort-order-selected").click()

driver.find_element(By.CSS_SELECTOR, ".default-message:nth-child(3)").click()

driver.find_element(By.CSS_SELECTOR, "#\\{\\\"value\\\"\\3A\\\"totreturns\\\"\\,\\\"label\\\"\\3A\\\"Current\\ Returns\\ \\(\\%\\)\\\"\\,\\\"id\\\"\\3A\\\"current_returns\\\"\\} > .label").click()

driver.find_element(By.CSS_SELECTOR, ".b-standard-select-return-as").click()

driver.find_element(By.CSS_SELECTOR, "#\\{\\\"value\\\"\\3A\\\"Absolute\\\"\\,\\\"label\\\"\\3A\\\"Absolute\\ \\%\\\"\\,\\\"key\\\"\\3A\\\"totreturns\\\"\\} > .label").click()
#  till here sorting etc is done

mf_returns = []

for item in driver.find_elements(By.CSS_SELECTOR, ".b-portfolio-fund-info__return-positive-desktop"):
    abs_ret = item.get_attribute('innerText')
    if (float(abs_ret[:-1]) < 10):
        break
    mf_returns.append(abs_ret)
    
number_of_funds = len(mf_returns)

# if number_of_funds == 0:
#     print("No Funds with over 10% absolute returns")
#     return

mf_names = []

for item in driver.find_elements(By.CSS_SELECTOR, ".b-portfolio-fund-info__name"):
    if len(mf_names) == number_of_funds:
        break
    mf_names.append(item.get_attribute('innerText'))
    
# print(*tuple(zip(mf_names, mf_returns)), sep = '\n') # this is just to display fundwise absolute returns
# till here we have the funds with greater than 10% absolute returns

for fund in mf_names:
      
    driver.find_element(By.XPATH, f"//*[contains(text(), '{fund}')]").click()
    #Now we are inside fund page
            
    driver.execute_script("window.scrollTo(0, 0);")
#     print(fund)
    
    time.sleep(.5)
    flag = 0
    while flag == 0:
        try:
            driver.find_element(By.XPATH, "//*[contains(text(), 'Transactions')]").click()
            total_invested_value = driver.find_element(By.CSS_SELECTOR, ".b-fund-detail-desktop__info__value__amount").get_attribute('innerText')
            current_NAV_of_fund = float(driver.find_element(By.CSS_SELECTOR, ".b-fund-detail-desktop__subtext__nav__value").get_attribute('innerText'))
            flag = 1
        except ElementClickInterceptedException as e:
            driver.execute_script("window.scrollTo(0, 0);")

    total_invested_value = float(total_invested_value[1:].replace(",", ""))
    total_units = total_invested_value / current_NAV_of_fund
    
    #logic to register transactions
    Invested_row = []
    Units_row = []
    NAV_row = []
    
    count = 0
    for amt in driver.find_elements(By.CSS_SELECTOR, ".b-transactions-desktop__item__amount__value"):
        if count % 2 == 0:
            amount = amt.get_attribute('innerText')
            try:
                amount = float(amount[1:].replace(",", ""))
            except ValueError as e:
                amount = 0
            Invested_row.append(amount)
        count += 1
        
    for amt in driver.find_elements(By.CSS_SELECTOR, ".b-transactions-desktop__item__date-section__info__units"):
        Units_row.append(float(amt.get_attribute('innerText')))
        
    for amt in driver.find_elements(By.CSS_SELECTOR, ".b-transactions-desktop__item__date-section__info__nav"):
        NAV_row.append(float(amt.get_attribute('innerText')))
        
    #logic to remove redemption transactions
    for i in range(len(Invested_row) - 1, -1, -1):
        if Invested_row[i] == 0:
            del Invested_row[i]
            del Units_row[i]
            del NAV_row[i]
            
    #logic to remove already redeemed transactions
    units_sum_check = total_units
    for i in range(len(Units_row)):
        if units_sum_check > Units_row[i]:
            units_sum_check -= Units_row[i]
        else:
            Units_row[i] = units_sum_check
            Units_row = Units_row[:i + 1]
            NAV_row = NAV_row[:i + 1]
            break
            
    units_to_be_redeemed = 0        
    for i in reversed(range(len(NAV_row))):
        if current_NAV_of_fund > 1.1 * NAV_row[i]:
            units_to_be_redeemed += Units_row[i]
        else:
            break
    
    if units_to_be_redeemed > 0:
        print(fund, units_to_be_redeemed)    
    driver.back()
    
driver.quit()

Canara Robeco Equity Tax Saver Growth Direct Plan 200.6668085408243
Mirae Asset Tax Saver Growth Direct Plan 732.188
Quant Tax Growth Direct Plan 956.118
Axis Long Term Equity Growth Direct Plan 298.8753890952907
ICICI Prudential Pharma Healthcare & Diagnostics Growth Direct Plan 45.898620232172505
Mirae Asset Emerging Bluechip Growth Direct Plan 0.0020972316002705327
DSP Healthcare Growth Direct Plan 0.014480503519301635
Quant Mid Cap Growth Direct Plan 51.46471059013547
Axis Mid Cap Growth Direct Plan 0.010937769490746874
Canara Robeco Bluechip Equity Growth Direct Plan 0.017388654656954827
 Edelweiss Europe Dynamic Equity Off Shore Growth Direct Plan 30.390092206390886
Aditya Birla Sunlife Life Pharma & Healthcare Growth Direct Plan 53.41930816422837
HDFC Nifty 100 Equal Weight Index Growth Direct Plan 0.005969093521471791
Motilal Oswal S&P 500 Index Growth Direct Plan 53.81983092608324
PGIM India Midcap Opportunities Growth Direct Plan 0.008967935496020374
 Nippon India Taiwan Equi